___
## Ciência dos Dados - PROJETO 3 - INSPER

___
#### Victor Arruda Niubó

#### Giovanni Pertence Cardoso dos Santos

#### Gustavo Schlieper Tessitore

#### Leonardo Cesari Costa
___

# Previsão da quantidade de views

___

## 1. INTRODUÇÃO

O objetivo do projeto é estimar a quantidade de visualizações que um vídeo terá com base no título, horário de publicação e as tags atribuidas aos vídeos.

___
## 2. MINERANDO DADOS e CARACTERÍSTICAS DO DATASET

In [19]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy.stats
import itertools
import math
import json
import os
from scipy.stats import norm

In [20]:
#remove mesagens de aviso do Pandas
pd.options.mode.chained_assignment = None
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
/Users/giovannid.santos/Documents/Insper/3_Semestre/CDADOS-DP/cdados_dp/P3


### BASE DE DADOS

In [21]:
# Base de dados dos videos em destaque do youtube US
you_data = pd.read_csv("USvideos.csv")

you_data.drop_duplicates(subset = 'video_id', keep = 'last', inplace = True)

# jason que contém a relação id - categoria
ids = json.loads(open('US_category_id.json').read())


# filtro das variáveis de interesse
you_analysis = you_data.loc[you_data.views|you_data.title|you_data.tags|you_data.category_id,\
                           ['views', 'title', 'tags', 'category_id']]


In [22]:
# separação da string das tags em uma lista de strings
tags_split = you_analysis.tags.str.split(pat = '|')
you_analysis['tags_split'] = tags_split
you_analysis.head()

,views,title,tags,category_id,tags_split
10,81377,Dion Lewis' 103-Yd Kick Return TD vs. Denver! ...,NFL|Football|offense|defense|afc|nfc|American ...,17,"[NFL, Football, offense, defense, afc, nfc, Am..."
36,288922,TL;DW - Every DCEU Movie Before Justice League,screenjunkies|screen junkies|sj news|honest tr...,1,"[screenjunkies, screen junkies, sj news, hones..."
41,34785,Iraq-Iran earthquake: Deadly tremor hits borde...,bbc|bbc news|news|iran|iran news|iraq|iraq new...,25,"[bbc, bbc news, news, iran, iran news, iraq, i..."
55,1496225,Jason Momoa Wows Hugh Grant With Some Dothraki...,Graham Norton|Graham Norton Show Official|Ente...,24,"[Graham Norton, Graham Norton Show Official, E..."
76,237307,Mayo Clinic's first face transplant patient me...,Mayo Clinic|Health Care (Issue)|Healthcare Sci...,28,"[Mayo Clinic, Health Care (Issue), Healthcare ..."


In [23]:
# Junta todas as strings em uma lista só para pder fazer um value_counts() e criar um
# dataframe com as tags e as frquencias de cada uma
merged = list(itertools.chain(*you_analysis.tags_split))
contador = pd.DataFrame({'col':merged})
limpino = contador[contador.col != '[none]']
dfl = limpino.col.value_counts().rename_axis('tags').reset_index(name='counts')
dfl = dfl[(dfl['counts'])> 10]

In [24]:
dfl

,tags,counts
0,funny,631
1,comedy,515
2,humor,277
3,comedian,227
4,music,213
5,celebrities,212
6,interview,203
7,how to,197
8,2018,197
9,celebrity,195


In [26]:
# Separa as linhas que contém certa tag

for e in range(len(dfl.tags)):
#     var_df = you_analysis[you_analysis['tags'].str.contains(dfl.tags[e])]
#     var_df[e + '_freq'] = dfl.counts[contador]
#     #var_df[e + '_freq'] =  var_df[e + '_freq'].astype('int')

#     not_var_df = you_analysis[~you_analysis['tags'].str.contains(e)]
#     not_var_df[e + '_freq'] = 0
#     #not_var_df[e + '_freq'] =  not_var_df[e + '_freq'].astype('int')
    
#     contador += 1
    
#     toto = pd.concat([var_df, not_var_df])
#     you_analysis[e + '_freq'] = toto[e + '_freq']
    
    var_df = you_analysis[you_analysis['tags'].str.contains(dfl.tags[e])]
    x = list(var_df.index)
    you_analysis[dfl.tags[e]+'_freq'] = 0
    you_analysis.loc[x,[dfl.tags[e]+'_freq']] = dfl.counts[contador]

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:17: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.


In [25]:
var_df = you_analysis[you_analysis['tags'].str.contains('funny')]
x = list(var_df.index)
you_analysis['funny_freq'] = 0
you_analysis.loc[x,['fun_freq']] = 1
you_analysis

,views,title,tags,category_id,tags_split,fun_freq
10,81377,Dion Lewis' 103-Yd Kick Return TD vs. Denver! ...,NFL|Football|offense|defense|afc|nfc|American ...,17,"[NFL, Football, offense, defense, afc, nfc, Am...",0
36,288922,TL;DW - Every DCEU Movie Before Justice League,screenjunkies|screen junkies|sj news|honest tr...,1,"[screenjunkies, screen junkies, sj news, hones...",0
41,34785,Iraq-Iran earthquake: Deadly tremor hits borde...,bbc|bbc news|news|iran|iran news|iraq|iraq new...,25,"[bbc, bbc news, news, iran, iran news, iraq, i...",0
55,1496225,Jason Momoa Wows Hugh Grant With Some Dothraki...,Graham Norton|Graham Norton Show Official|Ente...,24,"[Graham Norton, Graham Norton Show Official, E...",0
76,237307,Mayo Clinic's first face transplant patient me...,Mayo Clinic|Health Care (Issue)|Healthcare Sci...,28,"[Mayo Clinic, Health Care (Issue), Healthcare ...",0
78,1497519,Jason Momoa & Lisa Bonet: Love at First Sight,James Corden|The Late Late Show|Colbert|late n...,24,"[James Corden, The Late Late Show, Colbert, la...",1
102,225286,Mark Wahlberg's Kids Use Him for His Celeb Con...,Late Night|Seth Meyers|Mark Wahlberg|Kids|Cele...,23,"[Late Night, Seth Meyers, Mark Wahlberg, Kids,...",1
104,5476737,Camila Cabello - Havana (Vertical Video) ft. Y...,camila cabello|camila|young thug|havana|fifth ...,10,"[camila cabello, camila, young thug, havana, f...",0
106,304926,"Owen Wilson on Meeting Julia Roberts, and Tand...",owen wilson|owen|wilson|julia roberts|tandem b...,24,"[owen wilson, owen, wilson, julia roberts, tan...",0
117,1017803,Meryl Streep Meets Anna Wintour at Vogue,anna wintour|meryl streep|meryl streep intervi...,24,"[anna wintour, meryl streep, meryl streep inte...",0


In [9]:
banana = pd.DataFrame()
banana['views'] = you_analysis['views']
banana['top_freq'] = dfl.iloc[:,5:].max(axis=1)
banana
#plt.scatter(banana.top_freq, banana.views)

#you_analysis[(you_analysis['views'])> 200000000]

,views,top_freq
10,81377,NaN
36,288922,NaN
41,34785,NaN
55,1496225,NaN
76,237307,NaN
78,1497519,NaN
102,225286,NaN
104,5476737,NaN
106,304926,NaN
117,1017803,NaN


In [10]:
#trans.rename(index=str, columns={e: "vid_{0}".format(e)})
trans.dtypes

NameError: name 'trans' is not defined

In [ ]:
trans.nlargest(3,17)

In [ ]:
for index, row in trans.iterrows():
     print(index)

### ANÁLISE DESCRITIVA

___
## C. MODELOS DE PREDIÇÃO

o MODELO DE PREDIÇÃO PELA MÉDIA (Sem uso de variável explicativa).

o MODELO DOS K VIZINHOS MAIS PRÓXIMOS (K-Nearest Neighbors Regression)

o MODELO DE REGRESSÃO LINEAR (Multiple Linear Regression)

o MODELO DE ÁRVORES DE REGRESSÃO (Decision Tree Regression)

___
## D. PROCESSO E ESTATÍSTICAS DE VALIDAÇÃO

[Esse item depende dos resultados das modelagens anteriores! Organize-os aqui de forma clara!]

___
## E. CONCLUSÃO

___
## F. REFERÊNCIAS BIBLIOGRÁFICAS